Questions:
# 1. We want to check the accuracy of a linear regression model (OLS) with bootstrap. Use
the College dataset, with Grad.Rate as the outcome variable, and Accept as the predictor.
Use 0 as seed everywhere to make it replicable, points will be deducted if not. 5pt.

Hint for how I structured this: 

define a bootstrap standard errors function, 

define a bootstrap OLS
function, define a partial of the bootstrap OLS function, 

use the bootstrap standard errors function with the partial bootstrap OLS function to get the bootstrap estimated SE, 

and lastly estimate a univariate OLS and use print(summarize(results)) to see the SE estimated by OLS.

Hint for what output should be:

intercept 0.783696

Accept 0.000180

coef std err t P>|t|

Intercept 64.5110 0.797 80.915 0.000

Accept 0.0005 0.000 1.878 0.061

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm


C:\Users\ekim298\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load data: college
import pandas as pd
# 3
df = pd.read_csv('College.csv')
df
print(df.columns)

Index(['Unnamed: 0', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc',
       'Top25perc', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board',
       'Books', 'Personal', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni',
       'Expend', 'Grad.Rate'],
      dtype='object')


In [3]:
accept_model = sklearn_sm(sm.OLS,
                      MS(['Accept']))
X, Y = df.drop(columns=['Grad.Rate']), df['Grad.Rate']
cv_results = cross_validate(accept_model,
                            X,
                            Y,
                            cv=df.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err
# default: 5-fold cross validation. ISLP.sklearn_sm.
# MS: ModelSpec developed by the authors. Automatically add constant term.

294.4774085689979

In [4]:
def boot_SE(func,
            Data,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or Data.shape[0]
    for _ in range(B):
        idx = rng.choice(Data.index,
                         n,
                         replace=True)
        value = func(Data, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [ ]:
# alpha_SE = boot_SE(alpha_func,
#                    Portfolio,
#                    B=1000,
#                    seed=0)
# alpha_SE

In [5]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [6]:
accept_func = partial(boot_OLS, MS(['Accept']), 'Grad.Rate')
# fix the first two arguments of boot_OLS

In [7]:
rng = np.random.default_rng(0)
np.array([accept_func(df,
          rng.choice(df.index,
                     392,
                     replace=True)) for _ in range(10)])


array([[ 6.51651592e+01,  2.33102323e-04],
       [ 6.32527952e+01,  5.49088126e-04],
       [ 6.29716177e+01,  7.96768931e-04],
       [ 6.30566053e+01,  5.80610400e-04],
       [ 6.35605670e+01,  7.15719559e-04],
       [ 6.46067733e+01,  1.74216373e-04],
       [ 6.49647352e+01,  5.37439256e-04],
       [ 6.53941110e+01,  5.38808057e-04],
       [ 6.51972080e+01,  3.76260375e-04],
       [ 6.53435441e+01, -2.91636119e-04]])

In [8]:
accept_se = boot_SE(accept_func,
                df,
                B=1000,
                seed=0)
accept_se

intercept    0.701273
Accept       0.000168
dtype: float64

In [10]:
accept_model.fit(df, df['Grad.Rate'])
model_se = summarize(accept_model.results_)['std err']
model_se
print(summarize(accept_model.results_))

              coef  std err       t  P>|t|
intercept  64.5110    0.797  80.915  0.000
Accept      0.0005    0.000   1.878  0.061
